In [6]:
import pickle
import numpy as np
import torch
import matplotlib.pyplot as plt
import os
import sys
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

MODEL_NAME = ".\\{}_{}_10_res_loss_10_power_features.pt".format("Positive","EPM")
EPM_TRAIN_DICT_PATH = ".\\EPM_model\\EPM_train_dict_May_17.pkl"
EPM_VAL_DICT_PATH = ".\\EPM_model\\EPM_val_dict_May_17.pkl"
OFT_TRAIN_FILE_NAME = ".\\OFT_model\\OFT_train_dict_old_features_hand_picked.pkl"
OFT_VAL_FILE_NAME = ".\\OFT_model\\OFT_validation_dict_old_features_hand_picked.pkl"
FLX_TRAIN_FILE = ".\\FLX_model\\FLX_train_dict_old_features.pkl"
FLX_VAL_FILE = ".\\FLX_model\\FLX_validation_dict_old_features.pkl"

FEATURE_LIST = ['X_power_1_2','X_coh_1_2','X_gc_1_2']
FEATURE_VECTOR = FEATURE_LIST
FEATURE_WEIGHTS = [10,1,1]
NETWORK_CONSTRAINT = "Positive"
UMC_PATH = ".\\Universal-Mouse-Code\\"

sys.path.append(UMC_PATH)
#from dCSFA_model import dCSFA_model
import umc_data_tools as umc_dt
from dCSFA_NMF import dCSFA_NMF

if torch.cuda.is_available():
    device="cuda:0"
else:
    device="cpu"


print("Using device: %s"%(device))

#For Consistency
RANDOM_STATE=42

import pandas as pd

model = torch.load(MODEL_NAME,map_location='cpu')
model.device = "cpu"

Using device: cuda:0


In [3]:
#We aren't working with the test data yet
with open(EPM_TRAIN_DICT_PATH,"rb") as f:
    train_dict = pickle.load(f)

with open(EPM_VAL_DICT_PATH,"rb") as f:
    validation_dict = pickle.load(f)

#scale_vector = np.array([np.arange(1,57) for feature in range(64)]).flatten()
#Load the data
running_idx = 0
feature_groups = []
for idx,feature in enumerate(FEATURE_VECTOR):
    f_begin = running_idx
    f_end = f_begin + train_dict[feature].shape[1] 
    if idx == 0:
        f_end = f_end -1
    feature_groups.append((f_begin,f_end))

    running_idx = f_end

X_train = np.hstack([train_dict[feature]*weight for feature,weight in zip(FEATURE_VECTOR,FEATURE_WEIGHTS)])
y_roi_train = ~(train_dict['y_ROI']%2).astype(bool)
y_hc_train = train_dict['y_Homecage'].astype(bool)
y_vel_train = np.array(train_dict['y_vel'])
y_mouse_train = train_dict['y_mouse']
y_time_train = train_dict['y_time']

train_roi_nan_mask = (train_dict['y_ROI'] > 0)
train_in_hc_mask = y_hc_train
train_in_task_mask = ~y_hc_train
train_velocity_gt_8_mask = np.ones_like(y_time_train)#(y_vel_train > 0)

train_with_hc_mask = train_velocity_gt_8_mask
#train_with_hc_mask = np.logical_and(train_roi_nan_mask,train_velocity_gt_8_mask)
train_in_task_mask = np.logical_and(train_with_hc_mask,train_in_task_mask)


X_task_train = X_train[train_with_hc_mask==1]
y_hc_task_train = ~y_hc_train[train_with_hc_mask==1]
y_mouse_task_train = y_mouse_train[train_with_hc_mask==1]
y_roi_task_train = y_roi_train[train_with_hc_mask==1]

X_validation = np.hstack([validation_dict[feature]*weight for feature,weight in zip(FEATURE_VECTOR,FEATURE_WEIGHTS)])
y_roi_validation = ~(validation_dict['y_ROI']%2).astype(bool)
y_hc_validation = validation_dict['y_Homecage'].astype(bool)
y_vel_validation = validation_dict['y_vel']
y_mouse_validation = validation_dict['y_mouse']
y_time_validation = validation_dict['y_time']
validation_roi_nan_mask = (validation_dict['y_ROI'] > 0)
validation_in_task_mask = ~y_hc_validation
validation_in_hc_mask = y_hc_validation
validation_velocity_gt_8_mask = np.ones_like(y_time_validation)#(y_vel_validation > 0)

validation_with_hc_mask = np.logical_and(validation_roi_nan_mask,validation_velocity_gt_8_mask)
validation_with_hc_mask = validation_velocity_gt_8_mask
validation_in_task_mask = np.logical_and(validation_with_hc_mask,validation_in_task_mask)

X_task_val = X_validation[validation_with_hc_mask==1]
y_hc_task_val = ~y_hc_validation[validation_with_hc_mask==1]
y_mouse_task_val = y_mouse_validation[validation_with_hc_mask==1]
y_roi_task_val = y_roi_validation[validation_with_hc_mask==1]

intercept_mask = OneHotEncoder().fit_transform(y_mouse_task_train.reshape(-1,1)).todense()

In [7]:
y_pred, s = model.predict(X_task_train,include_scores=True)
roc_mean,roc_std = umc_dt.get_mean_std_err_auc(y_pred,y_hc_task_train,y_mouse_task_train)
print("train roc auc mean: %0.2f +/- %0.2f"%(roc_mean,roc_std))
mw_mean,mw_std = umc_dt.get_mean_std_err_auc(y_pred,y_hc_task_train,y_mouse_task_train,s,True)
print("train mw auc mean: %0.2f +/- %0.2f"%(mw_mean,mw_std))
y_pred, s = model.predict(X_task_val,include_scores=True)
roc_mean,roc_std = umc_dt.get_mean_std_err_auc(y_pred,y_hc_task_val,y_mouse_task_val)
print("val roc auc mean: %0.2f +/- %0.2f"%(roc_mean,roc_std))
mw_mean,mw_std = umc_dt.get_mean_std_err_auc(y_pred,y_hc_task_val,y_mouse_task_val,s,True)
print("val mw auc mean: %0.2f +/- %0.2f"%(mw_mean,mw_std))

y_pred, s = model.predict(X_task_train,include_scores=True)
roc_auc_dict = umc_dt.lpne_auc(y_pred,y_hc_task_train,y_mouse_task_train)
mw_auc_dict = umc_dt.lpne_auc(y_pred,y_hc_task_train,y_mouse_task_train,s,True)
print("train roc auc dict: ",roc_auc_dict)
print("train mw auc dict: ",mw_auc_dict)
df = umc_dt.make_projection_csv(EPM_TRAIN_DICT_PATH,model,FEATURE_VECTOR,
                                ['y_mouse','y_expDate','y_time','y_Homecage','y_ROI','y_vel','y_locx','y_locy'],
                                save_file =".\\July_30_Projections\\{}_EPM_onto_EPM_Train_Projection_2022_July_30.csv".format(NETWORK_CONSTRAINT),
                                auc_dict=mw_auc_dict)

y_pred, s = model.predict(X_task_val,include_scores=True)
roc_auc_dict = umc_dt.lpne_auc(y_pred,y_hc_task_val,y_mouse_task_val)
mw_auc_dict = umc_dt.lpne_auc(y_pred,y_hc_task_val,y_mouse_task_val,s,True)
print("validation roc auc dict: ",roc_auc_dict)
print("validation mw auc dict: ",mw_auc_dict)

df = umc_dt.make_projection_csv(EPM_VAL_DICT_PATH,model,FEATURE_VECTOR,
                                ['y_mouse','y_expDate','y_time','y_Homecage','y_ROI','y_vel','y_locx','y_locy'],
                                save_file =".\\July_30_Projections\\{}_EPM_onto_EPM_Val_Projection_2022_July_30.csv".format(NETWORK_CONSTRAINT),
                                auc_dict=mw_auc_dict)

train roc auc mean: 0.99 +/- 0.00
train mw auc mean: 0.99 +/- 0.00
val roc auc mean: 0.69 +/- 0.03
val mw auc mean: 0.81 +/- 0.02
train roc auc dict:  {'auc_method': 'sklearn_roc_auc', 'Mouse04201': 0.9952229299363058, 'Mouse04202': 0.9984662576687117, 'Mouse04205': 0.9444444444444444, 'Mouse04215': 0.9928977272727273, 'Mouse0630': 0.9991708126036485, 'Mouse0634': 0.9842105263157894, 'Mouse0643': 0.9940828402366864, 'Mouse1551': 0.9885145482388974, 'Mouse39114': 0.9873737373737373, 'Mouse39124': 1.0, 'Mouse39133': 1.0, 'Mouse6291': 0.9991909385113269, 'Mouse6292': 0.9992050874403815, 'Mouse6293': 0.9959677419354839, 'Mouse69064': 0.9982876712328768, 'Mouse69074': 1.0, 'Mouse8580': 0.9863387978142076, 'Mouse8581': 0.9807256235827664, 'Mouse8582': 0.9285714285714286, 'Mouse8891': 0.9975083056478405, 'Mouse8894': 0.9879032258064516}
train mw auc dict:  {'auc_method': 'mannWhitneyU', 'Mouse04201': (0.9951085090964568, 2.205912216434579e-105), 'Mouse04202': (0.9999450599761927, 1.2919215240

### FLX Projections

In [8]:
with open(FLX_TRAIN_FILE,"rb") as f:
    flx_train_dict = pickle.load(f)

with open(FLX_VAL_FILE,"rb") as f:
    flx_validation_dict = pickle.load(f)

FULL_FEATURE_LIST = ["X_power_1_2_full","X_coh_1_2_full","X_gc_1_2_full"]

flx_X_train = np.hstack([flx_train_dict[feature]*weight for feature,weight in zip(FEATURE_VECTOR,FEATURE_WEIGHTS)])
flx_y_train = flx_train_dict['y_flx']
flx_y_mouse_train = flx_train_dict['y_mouse']
flx_y_expDate_train = flx_train_dict['y_expDate']
flx_y_time_train = flx_train_dict['y_time']

flx_X_validation = np.hstack([flx_validation_dict[feature]*weight for feature,weight in zip(FEATURE_VECTOR,FEATURE_WEIGHTS)])
flx_y_validation = flx_validation_dict['y_flx']
flx_y_mouse_validation = flx_validation_dict['y_mouse']
flx_y_expDate_validation = flx_validation_dict['y_expDate']
flx_y_time_validation = flx_validation_dict['y_time']

flx_X = np.vstack([flx_X_train,flx_X_validation])
flx_y_task = np.hstack([flx_y_train,flx_y_validation])
flx_y_mouse = np.hstack([flx_y_mouse_train,flx_y_mouse_validation])
flx_y_expDate = np.hstack([flx_y_expDate_train,flx_y_expDate_validation])
flx_y_time = np.hstack([flx_y_time_train,flx_y_time_validation])

y_pred, s = model.predict(flx_X,include_scores=True)
roc_auc_dict = umc_dt.lpne_auc(y_pred,flx_y_task,flx_y_mouse)
print("ROC AUCs: ",roc_auc_dict)
mw_auc_dict = umc_dt.lpne_auc(y_pred,flx_y_task,flx_y_mouse,s,True)
print("MW AUCs: ",mw_auc_dict)

df = umc_dt.make_projection_csv(FLX_TRAIN_FILE,model,FULL_FEATURE_LIST,
                    ['y_mouse_full','y_expDate_full','y_time_full','y_flx_full'],
                    save_file = ".\\July_30_Projections\\{}_EPM_onto_FLX_flx_vs_saline_Train_Projection_2022_July_30.csv".format(NETWORK_CONSTRAINT),
                    auc_dict=mw_auc_dict,mousevar="y_mouse_full")

df = umc_dt.make_projection_csv(FLX_VAL_FILE,model,FULL_FEATURE_LIST,
                    ['y_mouse_full','y_expDate_full','y_time_full','y_flx_full'],
                    save_file = ".\\July_30_Projections\\{}_EPM_onto_FLX_flx_vs_saline_Val_Projection_2022_July_30.csv".format(NETWORK_CONSTRAINT),
                    auc_dict=mw_auc_dict,mousevar="y_mouse_full")

c:\Users\micha\Anaconda3\envs\cpne\lib\site-packages\sklearn\metrics\_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
c:\Users\micha\Anaconda3\envs\cpne\lib\site-packages\sklearn\metrics\_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(


ROC AUCs:  {'auc_method': 'sklearn_roc_auc', 'Mouse3191': 0.5131899994641982, 'Mouse3192': 0.6566808865352555, 'Mouse3193': 0.4020916811355944, 'Mouse3194': 0.3934731830963697, 'Mouse3202': nan, 'Mouse3203': 0.5095354303785321, 'Mouse99003': nan}
Mouse  Mouse3202  has only one class - AUC cannot be calculated
n_positive samples  1504
n_negative samples  0
Mouse  Mouse99003  has only one class - AUC cannot be calculated
n_positive samples  1874
n_negative samples  0
MW AUCs:  {'auc_method': 'mannWhitneyU', 'Mouse3191': (0.5211005841417955, 0.026575407787861396), 'Mouse3192': (0.8033684846033147, 5.240451088883674e-217), 'Mouse3193': (0.3423598695972237, 1.695419428563834e-60), 'Mouse3194': (0.30396394695297496, 1.0246819820737664e-93), 'Mouse3203': (0.4941479650098401, 0.5439473304515186)}


In [9]:
y_pred, s = model.predict(flx_X,include_scores=True)
roc_mean,roc_std = umc_dt.get_mean_std_err_auc(y_pred,flx_y_task,flx_y_mouse)
mw_mean,mw_std = umc_dt.get_mean_std_err_auc(y_pred,flx_y_task,flx_y_mouse,s,True)
print("EPM->FLX roc-auc: {} +/- {}".format(roc_mean,roc_std))
print("EPM->FLX mw-auc: {:0.2f} +/- {:0.2f}".format(mw_mean,mw_std))
#umc_dt.make_recon_plots(model,flx_X,0,task="EPM onto FLX (FLX vs Saline) Train and Val",saveFile="epm_onto_flx_recon.png")

Mouse  Mouse3202  has only one class - AUC cannot be calculated
n_positive samples  1504
n_negative samples  0
Mouse  Mouse99003  has only one class - AUC cannot be calculated
n_positive samples  1874
n_negative samples  0
EPM->FLX roc-auc: 0.49499423612199 +/- 0.047761131679591055
EPM->FLX mw-auc: 0.49 +/- 0.09


c:\Users\micha\Anaconda3\envs\cpne\lib\site-packages\sklearn\metrics\_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
c:\Users\micha\Anaconda3\envs\cpne\lib\site-packages\sklearn\metrics\_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(


## OFT Projections

In [10]:
with open(OFT_TRAIN_FILE_NAME,'rb') as f:
    train_dict = pickle.load(f)

with open(OFT_VAL_FILE_NAME,'rb') as f:
    val_dict = pickle.load(f)

NUM_FREQS = 56
NUM_FEATURES = np.hstack([train_dict[feature]*weight for feature,weight in zip(FEATURE_VECTOR,FEATURE_WEIGHTS)]).shape[1] // NUM_FREQS

#Train Arrays
oft_X_train = np.hstack([train_dict[feature]*weight for feature,weight in zip(FEATURE_VECTOR,FEATURE_WEIGHTS)])
oft_y_hc_train = train_dict['y_Homecage'].astype(bool)
oft_y_task_train = ~oft_y_hc_train
oft_y_ROI_train = train_dict['y_ROI']
oft_y_vel_train = train_dict['y_vel']
oft_y_mouse_train = train_dict['y_mouse']
oft_y_time_train = train_dict['y_time']

#Validation Arrays
oft_X_val = np.hstack([val_dict[feature]*weight for feature,weight in zip(FEATURE_VECTOR,FEATURE_WEIGHTS)])
oft_y_hc_val = val_dict['y_Homecage'].astype(bool)
oft_y_task_val = ~oft_y_hc_val
oft_y_ROI_val = val_dict['y_ROI']
oft_y_vel_val = val_dict['y_vel']
oft_y_mouse_val = val_dict['y_mouse']
oft_y_time_val = val_dict['y_time']

oft_X = np.vstack([oft_X_train,oft_X_val])
oft_y_task = np.hstack([oft_y_task_train,oft_y_task_val])
oft_y_mouse = np.hstack([oft_y_mouse_train,oft_y_mouse_val])


y_pred, s = model.predict(oft_X,include_scores=True)
roc_auc_dict = umc_dt.lpne_auc(y_pred,oft_y_task,oft_y_mouse)
print("ROC AUCs: ",roc_auc_dict)
mw_auc_dict = umc_dt.lpne_auc(y_pred,oft_y_task,oft_y_mouse,s,True)
print("MW AUCs: ",mw_auc_dict)

df = umc_dt.make_projection_csv(OFT_TRAIN_FILE_NAME,model,FEATURE_VECTOR,
                    ['y_mouse','y_expDate','y_time','y_Homecage','y_ROI','y_vel'],
                    save_file = ".\\July_30_Projections\\{}_EPM_onto_OFT_HC_vs_Task_Train_Projection_2022_July_30.csv".format(NETWORK_CONSTRAINT),
                    auc_dict=mw_auc_dict)

df = umc_dt.make_projection_csv(OFT_VAL_FILE_NAME,model,FEATURE_VECTOR,
                    ['y_mouse','y_expDate','y_time','y_Homecage','y_ROI','y_vel'],
                    save_file = ".\\July_30_Projections\\{}_EPM_onto_OFT_HC_vs_Task_Val_Projection_2022_July_30.csv".format(NETWORK_CONSTRAINT),
                    auc_dict=mw_auc_dict)

c:\Users\micha\Anaconda3\envs\cpne\lib\site-packages\sklearn\metrics\_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
c:\Users\micha\Anaconda3\envs\cpne\lib\site-packages\sklearn\metrics\_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
c:\Users\micha\Anaconda3\envs\cpne\lib\site-packages\sklearn\metrics\_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
c:\Users\micha\Anaconda3\envs\cpne\lib\site-packages\sklearn\metrics\_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(


ROC AUCs:  {'auc_method': 'sklearn_roc_auc', 'Mouse04191': 0.6366091932387155, 'Mouse04193': 0.7554046713742533, 'Mouse04201': 0.7927075731207546, 'Mouse04202': 0.762163150835264, 'Mouse04205': 0.7693898377913847, 'Mouse04215': nan, 'Mouse3191': 0.4798985035489851, 'Mouse3192': nan, 'Mouse3193': 0.5555724336424688, 'Mouse3194': nan, 'Mouse3203': nan, 'Mouse39114': 0.6642295597484278, 'Mouse39124': 0.7445234869015356, 'Mouse39125': 0.7472484276729559, 'Mouse39133': 0.7538408874654784, 'Mouse69064': 0.6018158488218369, 'Mouse69065': 0.6779515585967198, 'Mouse69072': 0.5848691788711535, 'Mouse69074': 0.6767914438502673}
Mouse  Mouse04215  has only one class - AUC cannot be calculated
n_positive samples  506
n_negative samples  0
Mouse  Mouse3192  has only one class - AUC cannot be calculated
n_positive samples  279
n_negative samples  0
Mouse  Mouse3194  has only one class - AUC cannot be calculated
n_positive samples  324
n_negative samples  0
Mouse  Mouse3203  has only one class - AUC c

In [11]:
y_pred, s = model.predict(oft_X,include_scores=True)
roc_mean,roc_std = umc_dt.get_mean_std_err_auc(y_pred,oft_y_task,oft_y_mouse)
mw_mean,mw_std = umc_dt.get_mean_std_err_auc(y_pred,oft_y_task,oft_y_mouse,s,True)
print("EPM->OFT roc-auc: {} +/- {}".format(roc_mean,roc_std))
print("EPM->OFT mw-auc: {:0.2f} +/- {:0.2f}".format(mw_mean,mw_std))
#umc_dt.make_recon_plots(model,oft_X,0,task="EPM onto OFT (HC vs Task) Train and Val",saveFile="epm_onto_oft_recon.png")

Mouse  Mouse04215  has only one class - AUC cannot be calculated
n_positive samples  506
n_negative samples  0
Mouse  Mouse3192  has only one class - AUC cannot be calculated
n_positive samples  279
n_negative samples  0
Mouse  Mouse3194  has only one class - AUC cannot be calculated
n_positive samples  324
n_negative samples  0
Mouse  Mouse3203  has only one class - AUC cannot be calculated
n_positive samples  500
n_negative samples  0
EPM->OFT roc-auc: 0.6802010503653466 +/- 0.0239144419711592
EPM->OFT mw-auc: 0.77 +/- 0.03


c:\Users\micha\Anaconda3\envs\cpne\lib\site-packages\sklearn\metrics\_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
c:\Users\micha\Anaconda3\envs\cpne\lib\site-packages\sklearn\metrics\_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
c:\Users\micha\Anaconda3\envs\cpne\lib\site-packages\sklearn\metrics\_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
c:\Users\micha\Anaconda3\envs\cpne\lib\site-packages\sklearn\metrics\_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
